In [1]:
import d3rlpy
import torch
import numpy as np
from matplotlib import pyplot as plt
from data.makeSeaquestdata import load_seaquest_dataset, make_seaquest_testset
from data.makeHalfcheetahdata import load_halfcheetah_dataset, make_halfcheetah_testset
from load_model import load_seaquest_model, load_halfcheetah_model
from utils import create_trajectories, get_trajectory_embedding, perform_clustering_and_plot, trajectory_attributions_hc, set_device_seeding, print_results_hc
from model import make_episodes, fit_discrete_sac, fit_per_cluster, fit_sac
from encoder import CustomCNNFactory

No module named 'flow'
No module named 'carla'


In [2]:
device = set_device_seeding(seed=0)

In [3]:
halfcheetahdata, hc_env = load_halfcheetah_dataset(env_name='halfcheetah-medium-v2', size=500*100, seed=0)
pre_trained_encoder_halfcheetah = load_halfcheetah_model("trajectory_transformer/logs/halfcheetah-medium-v2/gpt/pretrained", seed=0)
halfcheetahdata, test_observation_hc = make_halfcheetah_testset(halfcheetahdata, test_size=200)


# HalfCheetah data
final_obs_hc, final_act_hc, final_rew_hc, final_ter_hc = create_trajectories(
    halfcheetahdata["observations"], halfcheetahdata["actions"], 
    halfcheetahdata["rewards"], halfcheetahdata["terminals"], trajectory_length=25
)

print('Halfcheetah data obtained!')

trajectory_embedding_halfcheetah = get_trajectory_embedding(
    pre_trained_encoder_halfcheetah, final_obs_hc, final_act_hc, final_rew_hc, is_seaquest=False, device = device
)


c:\Users\tgome\Desktop\FACT\factvenv\lib\site-packages\gym\spaces\box.py:84: UserWarning: WARN: Box bound precision lowered by casting to float32
  logger.warn(f"Box bound precision lowered by casting to {self.dtype}")
load datafile: 100%|██████████| 21/21 [00:02<00:00,  7.34it/s]


Dataset extracted with 50000 samples.
Information about the dataset:
Observation shape:  (50000, 17)
Action shape:  (50000, 6)
Reward shape:  (50000,)
Action space:  (6,)
Observation space:  (17,)
Environment 'halfcheetah-medium-v2' initialized.


Processing trajectories:   0%|          | 0/1996 [00:00<?, ?it/s]c:\Users\tgome\Desktop\FACT\factvenv\lib\site-packages\torch\cuda\amp\autocast_mode.py:114: UserWarning: torch.cuda.amp.autocast only affects CUDA ops, but CUDA is not available.  Disabling.
  warnings.warn("torch.cuda.amp.autocast only affects CUDA ops, but CUDA is not available.  Disabling.")


Halfcheetah data obtained!


Processing trajectories: 100%|██████████| 1996/1996 [01:15<00:00, 26.42it/s]


In [4]:
clusters_halfcheetah, _ = perform_clustering_and_plot(trajectory_embedding_halfcheetah.detach().cpu().numpy(), 10, 10, ccore=True, plot=False)

In [5]:
list_episodes_hc = make_episodes(final_obs_hc, final_act_hc, final_rew_hc, 6)
sac_hc = fit_sac(list_episodes_hc, n_steps=100, n_steps_per_epoch=10, device=device)

model_params_hc = {
'actor_learning_rate': 3e-4,
'critic_learning_rate': 3e-4,
'temp_learning_rate': 3e-4,
'batch_size': 256,
'scaler': 'pixel',
'use_gpu': True if device == 'cuda' else False
}

models_hc, result_data_combinations_hc = fit_per_cluster(
    model=sac_hc, 
    model_class= d3rlpy.algos.SAC, 
    model_params=model_params_hc,
    data_embedding=trajectory_embedding_halfcheetah,
    list_episodes=list_episodes_hc, 
    clusters=clusters_halfcheetah, 
    trajectory_embedding=trajectory_embedding_halfcheetah,
    test_observations=np.expand_dims(test_observation_hc, axis=0))

2024-02-02 00:28.56 [debug    ] RandomIterator is selected.
2024-02-02 00:28.56 [info     ] Directory is created at d3rlpy_logs\SAC_20240202002856
2024-02-02 00:28.56 [debug    ] Fitting scaler...              scaler=pixel
2024-02-02 00:28.56 [debug    ] Building models...
2024-02-02 00:28.56 [debug    ] Models have been built.
2024-02-02 00:28.56 [info     ] Parameters are saved to d3rlpy_logs\SAC_20240202002856\params.json params={'action_scaler': None, 'actor_encoder_factory': {'type': 'default', 'params': {'activation': 'relu', 'use_batch_norm': False, 'dropout_rate': None}}, 'actor_learning_rate': 0.0003, 'actor_optim_factory': {'optim_cls': 'Adam', 'betas': (0.9, 0.999), 'eps': 1e-08, 'weight_decay': 0, 'amsgrad': False}, 'batch_size': 256, 'critic_encoder_factory': {'type': 'default', 'params': {'activation': 'relu', 'use_batch_norm': False, 'dropout_rate': None}}, 'critic_learning_rate': 0.0003, 'critic_optim_factory': {'optim_cls': 'Adam', 'betas': (0.9, 0.999), 'eps': 1e-08, 

Epoch 1/10: 100%|██████████| 10/10 [00:00<00:00, 15.63it/s, temp_loss=9.99, temp=1, critic_loss=147, actor_loss=-4.02]

2024-02-02 00:28.56 [info     ] SAC_20240202002856: epoch=1 step=10 epoch=1 metrics={'time_sample_batch': 0.0027399301528930665, 'time_algorithm_update': 0.0596752405166626, 'temp_loss': 10.039063358306885, 'temp': 0.9983514606952667, 'critic_loss': 143.48310546875, 'actor_loss': -4.2723876476287845, 'time_step': 0.06284396648406983} step=10
2024-02-02 00:28.57 [info     ] Model parameters are saved to d3rlpy_logs\SAC_20240202002856\model_10.pt



Epoch 2/10: 100%|██████████| 10/10 [00:00<00:00, 16.54it/s, temp_loss=10.1, temp=0.997, critic_loss=135, actor_loss=-4.51]

2024-02-02 00:28.57 [info     ] SAC_20240202002856: epoch=2 step=20 epoch=2 metrics={'time_sample_batch': 0.00291903018951416, 'time_algorithm_update': 0.0559603214263916, 'temp_loss': 10.055249500274659, 'temp': 0.9953596413135528, 'critic_loss': 127.53994522094726, 'actor_loss': -4.741487741470337, 'time_step': 0.05932912826538086} step=20
2024-02-02 00:28.57 [info     ] Model parameters are saved to d3rlpy_logs\SAC_20240202002856\model_20.pt


...

2024-02-02 00:30.17 [info     ] Model parameters are saved to d3rlpy_logs\SAC_20240202003012\model_80.pt


Epoch 9/10: 100%|██████████| 10/10 [00:00<00:00, 14.24it/s, temp_loss=9.69, temp=0.976, critic_loss=7.49, actor_loss=-11.4]

2024-02-02 00:30.18 [info     ] SAC_20240202003012: epoch=9 step=90 epoch=9 metrics={'time_sample_batch': 0.004046320915222168, 'time_algorithm_update': 0.06417675018310547, 'temp_loss': 9.616453266143798, 'temp': 0.9748219668865203, 'critic_loss': 7.413582420349121, 'actor_loss': -11.350332641601563, 'time_step': 0.06894483566284179} step=90
2024-02-02 00:30.18 [info     ] Model parameters are saved to d3rlpy_logs\SAC_20240202003012\model_90.pt



Epoch 10/10: 100%|██████████| 10/10 [00:00<00:00, 13.71it/s, temp_loss=9.57, temp=0.973, critic_loss=8.02, actor_loss=-11.4]


2024-02-02 00:30.18 [info     ] SAC_20240202003012: epoch=10 step=100 epoch=10 metrics={'time_sample_batch': 0.002778434753417969, 'time_algorithm_update': 0.06885135173797607, 'temp_loss': 9.501240921020507, 'temp': 0.9719759047031402, 'critic_loss': 7.593591833114624, 'actor_loss': -11.255508327484131, 'time_step': 0.07207462787628174} step=100
2024-02-02 00:30.18 [info     ] Model parameters are saved to d3rlpy_logs\SAC_20240202003012\model_100.pt


In [7]:

action_dict = {	0: 'bthigh',
            1: 'bshin',
            2: 'bfoot',
            3: 'fthigh',
            4: 'fshin',
            5: 'ffoot',            
}
#test_observation, models, traj_embeddings, clusters
#np.expand_dims(test_observation_hc, axis=0)
attributions_hc = trajectory_attributions_hc(np.expand_dims(test_observation_hc, axis=0), models_hc, trajectory_embedding_halfcheetah, clusters_halfcheetah)
print("Print table 2")
print_results_hc(result_data_combinations_hc, np.expand_dims(test_observation_hc, axis=0), models_hc, attributions_hc)

Print table 2
3.3615303
3.4558086
3.4691343
3.2958813
3.362166
3.3624597
3.5280466
3.3444352
3.320675
3.3745296
3.382636
Comparing the actions
[0.         0.00412198 0.0037523  0.00352279 0.00192288 0.00384305
 0.00279478 0.00163549 0.00526476 0.00283458 0.00136433]
Avg Delta Q
0.0
0.09427833557128906
0.1076040267944336
0.06564903259277344
0.0006356239318847656
0.0009293556213378906
0.16651630401611328
0.017095088958740234
0.04085540771484375
0.012999296188354492
0.02110576629638672
Data distances
[0.         1.         0.30475168 0.87304217 0.84830602 0.29862322
 0.53408937 0.52455791 0.61629598 0.60397571 0.78555106]
[0.1 0.3 0.  0.  0.6 0.  0.  0.  0.  0.  0. ]
